In [22]:
from dotenv import load_dotenv
from google.adk.agents.llm_agent import Agent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from google.adk.tools import agent_tool

In [23]:
load_dotenv(override=True)

True

In [24]:
gemini_api_key = os.getenv("GOOGLE_API_KEY")
gemini_llm_model = os.getenv("GEMINI_LLM_MODEL", "gemini-2.0-flash")

FROM_EMAIL = Email("r.sanjif@gmail.com")
TO_EMAIL = To("r.sanjif@hotmail.com")

In [25]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

sales_agent1 = Agent(
    name="Professional_Sales_Agent",
    model=gemini_llm_model,
    description=("You write professional, serious cold emails"),
    instruction=(instructions1),
)
sales_agent1_tool = agent_tool.AgentTool(agent=sales_agent1)  # Wrap the agent

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

sales_agent2 = Agent(
    name="Engaging_Sales_Agent",
    model=gemini_llm_model,
    description=(
        "You write witty, engaging cold emails that are likely to get a response"
    ),
    instruction=(instructions2),
)
sales_agent2_tool = agent_tool.AgentTool(agent=sales_agent2)  # Wrap the agent

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."
sales_agent3 = Agent(
    name="Busy_Sales_Agent",
    model=gemini_llm_model,
    description=("You write concise, to the point cold emails."),
    instruction=(instructions3),
)
sales_agent3_tool = agent_tool.AgentTool(agent=sales_agent3)  # Wrap the agent

In [26]:
best_email_picker_instructions = (
    "You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only."
)

best_email_picker = Agent(
    name="sales_picker",
    description=("You pick the best cold sales email from a set of provided emails"),
    instruction=(best_email_picker_instructions),
    model=gemini_llm_model,
)

best_email_picker_agent_tool = agent_tool.AgentTool(agent=best_email_picker)


In [27]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

subject_writer = Agent(
    name="Email_subject_writer",
    description=("You write the subject of the email based on the email body provided"),
    instruction=(subject_instructions),
    model=gemini_llm_model,
)

subject_writer_agent_tool = agent_tool.AgentTool(agent=subject_writer)

In [28]:
html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

html_converter = Agent(
    name="HTML_email_body_converter",
    description=("You convert the email into HTML format"),
    instruction=(html_instructions),
    model=gemini_llm_model,
)

html_converter_agent_tool = agent_tool.AgentTool(agent=html_converter)


In [29]:
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """Send out an email with the given subject and HTML body to all sales prospects"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get("SENDGRID_API_KEY"))
    from_email = FROM_EMAIL
    to_email = TO_EMAIL
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [30]:
tools = [
    sales_agent1_tool,
    sales_agent2_tool,
    sales_agent3_tool,
    best_email_picker_agent_tool,
    subject_writer_agent_tool,
    html_converter_agent_tool,
    send_html_email,
]

tools

 <function __main__.send_html_email(subject: str, html_body: str) -> Dict[str, str]>]

In [31]:
# Improved instructions thanks to student Guillermo F.

sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Use the email_picker_agent_tool to Review the drafts and choose the single best email using your judgment of which one is most effective.

3. Create the subject line: Use the subject_writer_agent_tool to write the subject of the email chosen.

4. Convert to html email format: Use the html_converter_agent_tool to convert the email to html format
 
5. Use the send_html_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""

sales_manager = Agent(
    name="Sales_Manager",
    instruction=(sales_manager_instructions),
    description=(
        "Your goal is to find the single best cold sales email using the sales_agent tools, pick the best email, get the subject of the email, convert to html format and send the email"
    ),
    tools=tools,
    model=gemini_llm_model,
)

In [32]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

session_service = InMemorySessionService()
APP_NAME = "Cold_Email_App"
USER_ID = "user_1"
SESSION_ID = "session_001"  # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

Session created: App='Cold_Email_App', User='user_1', Session='session_001'


In [33]:
# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=sales_manager,  # The agent we want to run
    app_name=APP_NAME,  # Associates runs with our app
    session_service=session_service,  # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Runner created for agent 'Sales_Manager'.


In [34]:
# @title Define Agent Interaction Function

from google.genai import types  # For creating message Content/Parts


async def call_agent_async(query: str, runner, user_id, session_id):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User Query: {query}")

    # Prepare the user's message in ADK format
    content = types.Content(role="user", parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response."  # Default

    # Key Concept: run_async executes the agent logic and yields Events.
    # We iterate through events to find the final answer.
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=content
    ):
        # You can uncomment the line below to see *all* events during execution
        print(
            f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}"
        )

        # Key Concept: is_final_response() marks the concluding message for the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assuming text response in the first part
                final_response_text = event.content.parts[0].text
            elif (
                event.actions and event.actions.escalate
            ):  # Handle potential errors/escalations
                final_response_text = (
                    f"Agent escalated: {event.error_message or 'No specific message.'}"
                )
            # Add more checks here if needed (e.g., specific error codes)
            break  # Stop processing events once the final response is found

    print(f"<<< Agent Response: {final_response_text}")

In [35]:
async def run_conversation():
    message = "Send out a cold sales email addressed to Dear CEO from Alice"
    await call_agent_async(
        message, runner=runner, user_id=USER_ID, session_id=SESSION_ID
    )

In [36]:
await run_conversation()


>>> User Query: Send out a cold sales email addressed to Dear CEO from Alice


  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  text="""Okay, I will generate three email drafts using the sales agent tools, then pick the best one, create a subject line, convert it to HTML format, and finally send the email.

First, I'll generate the drafts.

"""
), Part(
  function_call=FunctionCall(
    args={
      'request': 'Write a cold email addressed to Dear CEO from Alice'
    },
    id='adk-85793b13-5f87-4263-82eb-2d27d209ac53',
    name='Professional_Sales_Agent'
  )
)] role='model'
  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-85793b13-5f87-4263-82eb-2d27d209ac53',
    name='Professional_Sales_Agent',
    response={
      'result': """Subject: Streamlining SOC 2 Compliance with AI - [Your Company Name]

Dear CEO,

My name is Alice, and I'm reaching out from ComplAI. We understand that maintaining SOC 2 compliance can be a demanding and resource-intens

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'request': 'Write a cold email addressed to Dear CEO from Alice'
    },
    id='adk-45f4fd56-b5b5-4307-baeb-6032678c5dca',
    name='Engaging_Sales_Agent'
  )
)] role='model'
  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-45f4fd56-b5b5-4307-baeb-6032678c5dca',
    name='Engaging_Sales_Agent',
    response={
      'result': """Subject: SOC2 Compliance: So easy, it's practically cheating (but totally legal!)

Dear CEO,

Alice here, from ComplAI – the company that's making SOC2 compliance so ridiculously easy, it's almost unfair to your competitors who are still drowning in spreadsheets and auditor anxiety.

Look, I get it. SOC2 probably sounds about as appealing as a root canal performed with a rusty spork. But trust me, we've taken the "ugh" out of "audit."

With ComplAI, you 

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'request': 'Write a cold email addressed to Dear CEO from Alice'
    },
    id='adk-a6b12af0-85ae-41e0-a059-759bb9b1777b',
    name='Busy_Sales_Agent'
  )
)] role='model'
  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-a6b12af0-85ae-41e0-a059-759bb9b1777b',
    name='Busy_Sales_Agent',
    response={
      'result': """Subject: SOC2 Compliance Automation - [Company Name]

Dear CEO,

Is your team spending too much time on SOC2 compliance?

ComplAI automates the process, saving you time and resources while ensuring audit readiness.

Interested in a quick demo?

Best,

Alice
"""
    }
  )
)] role='user'


  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  text="""Now that I have the three drafts, I will evaluate them and pick the best one using the sales_picker tool.

"""
), Part(
  function_call=FunctionCall(
    args={
      'request': """Here are three cold emails:

Email 1: Subject: Streamlining SOC 2 Compliance with AI - [Your Company Name]

Dear CEO,

My name is Alice, and I'm reaching out from ComplAI. We understand that maintaining SOC 2 compliance can be a demanding and resource-intensive process for organizations like yours.

At ComplAI, we offer an AI-powered SaaS platform designed to significantly streamline SOC 2 compliance and prepare you for audits. Our platform automates evidence collection, simplifies policy management, and provides continuous monitoring to ensure ongoing compliance, ultimately reducing the time and cost associated with achieving and maintaining SOC 2 certification.

We believe ComplAI could be a valuable asset in enhanc

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  text="""The sales picker chose Email 1. Now I will create a subject line for it.

"""
), Part(
  function_call=FunctionCall(
    args={
      'request': """Write a subject line for the following email: Subject: Streamlining SOC 2 Compliance with AI - [Your Company Name]

Dear CEO,

My name is Alice, and I'm reaching out from ComplAI. We understand that maintaining SOC 2 compliance can be a demanding and resource-intensive process for organizations like yours.

At ComplAI, we offer an AI-powered SaaS platform designed to significantly streamline SOC 2 compliance and prepare you for audits. Our platform automates evidence collection, simplifies policy management, and provides continuous monitoring to ensure ongoing compliance, ultimately reducing the time and cost associated with achieving and maintaining SOC 2 certification.

We believe ComplAI could be a valuable asset in enhancing your security posture

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  text="""I will use option 1: "Simplify SOC 2 Compliance with AI: A 15-Minute Chat?"

Now, convert the email to HTML format.

"""
), Part(
  function_call=FunctionCall(
    args={
      'request': """Subject: Simplify SOC 2 Compliance with AI: A 15-Minute Chat?

Dear CEO,

My name is Alice, and I'm reaching out from ComplAI. We understand that maintaining SOC 2 compliance can be a demanding and resource-intensive process for organizations like yours.

At ComplAI, we offer an AI-powered SaaS platform designed to significantly streamline SOC 2 compliance and prepare you for audits. Our platform automates evidence collection, simplifies policy management, and provides continuous monitoring to ensure ongoing compliance, ultimately reducing the time and cost associated with achieving and maintaining SOC 2 certification.

We believe ComplAI could be a valuable asset in enhancing your security posture and simpl

  [Event] Author: Sales_Manager, Type: Event, Final: False, Content: parts=[Part(
  text="""Now I will send the email.

"""
), Part(
  function_call=FunctionCall(
    args={
      'html_body': """<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>Simplify SOC 2 Compliance with AI</title>
<style>
body {
  font-family: Arial, sans-serif;
  line-height: 1.6;
  color: #333;
  margin: 0;
  padding: 20px;
}
h1 {
  color: #0056b3;
  margin-bottom: 20px;
}
p {
  margin-bottom: 15px;
}
.cta-button {
  background-color: #4CAF50;
  color: white;
  padding: 12px 20px;
  text-decoration: none;
  border-radius: 5px;
  display: inline-block;
  margin-top: 20px;
}
.footer {
  margin-top: 30px;
  font-size: 0.8em;
  color: #777;
}
</style>
</head>
<body>

<h1>Simplify SOC 2 Compliance with AI: A 15-Minute Chat?</h1>

<p>Dear CEO,</p>

<p>My name is Alice, and I'm reaching out from ComplAI. We understand that maintaining SOC 2 compliance can be a demanding and resource-intensive process for org